In [1]:
import mlagents
from mlagents_envs.environment import UnityEnvironment as UE

In [2]:
env = UE(file_name="UnityEnvironment", seed=1, side_channels=[])

In [3]:
env.reset()

In [4]:
print(list(env.behavior_specs))
behavior_name = list(env.behavior_specs)[0]
print(behavior_name)

['3DBall?team=0']
3DBall?team=0


In [5]:
print(type(env.behavior_specs[behavior_name]))
print()
print(env.behavior_specs[behavior_name])
spec = env.behavior_specs[behavior_name]

<class 'mlagents_envs.base_env.BehaviorSpec'>

BehaviorSpec(observation_specs=[ObservationSpec(shape=(8,), dimension_property=(<DimensionProperty.UNSPECIFIED: 0>,), observation_type=<ObservationType.DEFAULT: 0>, name='')], action_spec=ActionSpec(continuous_size=2, discrete_branches=()))


In [6]:
print(type(spec.observation_specs))
print()
print(type(tuple(spec.observation_specs[0])))
print(spec.observation_specs[0])

<class 'list'>

<class 'tuple'>
ObservationSpec(shape=(8,), dimension_property=(<DimensionProperty.UNSPECIFIED: 0>,), observation_type=<ObservationType.DEFAULT: 0>, name='')


In [7]:
print(type(spec.action_spec))
print()
print(spec.action_spec)

<class 'mlagents_envs.base_env.ActionSpec'>

Continuous: 2, Discrete: ()


In [10]:
decision_steps, terminal_steps = env.get_steps(behavior_name)

In [11]:
print(decision_steps)
print(list(decision_steps))

[0]


In [12]:
tracked_agent = -1
done = False
episode_rewards = 0

print(type(decision_steps.agent_id))
tracked_agent = decision_steps.agent_id[0]

<class 'numpy.ndarray'>


In [13]:
action = spec.action_spec.random_action(len(decision_steps))
print(type(action.continuous))
print(action.continuous)
# action = action.continuous

<class 'numpy.ndarray'>
[[-0.98035    -0.04746135]]


In [14]:
env.set_actions(behavior_name, action)

In [15]:
for episode in range(30):
    env.reset()
    decision_steps, terminal_steps = env.get_steps(behavior_name)
    tracked_agent = -1 # -1 indicates not yet tracking
    done = False # For the tracked_agent
    episode_rewards = 0 # For the tracked_agent
    while not done:
        # Track the first agent we see if not tracking
        # Note : len(decision_steps) = [number of agents that requested a decision]
        if tracked_agent == -1 and len(decision_steps) >= 1:
            tracked_agent = decision_steps.agent_id[0]
        # Generate an action for all agents
        # action = spec.create_random_action(len(decision_steps))
        action = spec.action_spec.random_action(len(decision_steps))
        # Set the actions
        env.set_actions(behavior_name, action)
        # Move the simulation forward
        env.step()
        # Get the new simulation results
        decision_steps, terminal_steps = env.get_steps(behavior_name)
        if tracked_agent in decision_steps: # The agent requested a decision
            episode_rewards += decision_steps[tracked_agent].reward
#             print(decision_steps[tracked_agent].reward)
        if tracked_agent in terminal_steps: # The agent terminated its episode
            episode_rewards += terminal_steps[tracked_agent].reward
#             print(terminal_steps[tracked_agent].reward)
            done = True
    print(f"Total rewards for episode {episode} is {episode_rewards}")

Total rewards for episode 0 is 1.0000000298023224
Total rewards for episode 1 is 0.10000001639127731
Total rewards for episode 2 is 0.700000025331974
Total rewards for episode 3 is 0.700000025331974
Total rewards for episode 4 is 2.0000000447034836
Total rewards for episode 5 is 0.6000000238418579
Total rewards for episode 6 is 1.500000037252903
Total rewards for episode 7 is 1.1000000312924385
Total rewards for episode 8 is 0.40000002086162567
Total rewards for episode 9 is 0.30000001937150955
Total rewards for episode 10 is 0.8000000268220901
Total rewards for episode 11 is 0.6000000238418579
Total rewards for episode 12 is 0.6000000238418579
Total rewards for episode 13 is 1.1000000312924385
Total rewards for episode 14 is 1.7000000402331352
Total rewards for episode 15 is 0.6000000238418579
Total rewards for episode 16 is 0.5000000223517418
Total rewards for episode 17 is 1.8000000417232513
Total rewards for episode 18 is 2.7000000551342964
Total rewards for episode 19 is 0.9000000

In [79]:
env.close()